In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # '더보기' 클릭을 통한 리뷰 로드 반복
    while len(reviews) < total_reviews:
        # 모든 리뷰 요소를 가져와서 추가 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')

        # 각 리뷰 정보 수집
        for review_element in review_elements[len(reviews):]:  # 이미 수집한 리뷰는 건너뜁니다.
            try:
                # 작성자 이름 (출력용)
                author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                # 리뷰 개수 및 사진 개수
                try:
                    review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                except NoSuchElementException:
                    review_count_text = "Unknown"

                # 리뷰 내용과 날짜
                try:
                    # 사진이 있는 리뷰의 텍스트와 날짜
                    review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                    if review_text_elements:
                        review_text = review_text_elements[0].text.strip()
                    else:
                        # 누락된 리뷰 요소에 대한 새로운 경로
                        review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                except NoSuchElementException:
                    # 사진이 없는 리뷰의 텍스트와 날짜
                    review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                review_key = (author_name, review_text, review_date, review_count_text)
                if review_key not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add(review_key)
                    print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                    break
            except NoSuchElementException:
                continue

        # '더보기' 버튼이 없어지거나 리뷰가 더 이상 늘어나지 않으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
            driver.execute_script("window.scrollBy(0, 300);")  # 스크롤을 조금씩 내려 모든 리뷰 로드

        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (202

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # 10개씩 리뷰를 수집하고 '더보기' 버튼 클릭
    while len(reviews) < total_reviews:
        # 현재 리뷰를 최대 10개씩 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        
        for review_element in review_elements[len(reviews):len(reviews)+10]:  # 새로 10개씩 수집
            try:
                # 작성자 이름 (출력용)
                author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                # 리뷰 개수 및 사진 개수
                try:
                    review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                except NoSuchElementException:
                    review_count_text = "Unknown"

                # 리뷰 내용과 날짜
                try:
                    review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                    review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                    review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                except NoSuchElementException:
                    review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                review_key = (author_name, review_text, review_date, review_count_text)
                if review_key not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add(review_key)
                    print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                    break
            except NoSuchElementException:
                continue

        # '더보기' 버튼이 없어지거나 리뷰가 더 이상 늘어나지 않으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기

        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (202

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # 10개씩 리뷰를 수집하고 '더보기' 버튼 클릭
    while len(reviews) < total_reviews:
        # 리뷰 요소를 10개씩 재로드하여 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        
        for review_element in review_elements[len(reviews):len(reviews)+10]:  # 새로 10개씩 수집
            try:
                # 작성자 이름 (출력용)
                author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                # 리뷰 개수 및 사진 개수
                try:
                    review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                except NoSuchElementException:
                    review_count_text = "Unknown"

                # 리뷰 내용과 날짜
                try:
                    review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                    review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                    review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                except NoSuchElementException:
                    review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                review_key = (author_name, review_text, review_date, review_count_text)
                if review_key not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add(review_key)
                    print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                    break
            except NoSuchElementException:
                continue

        # '더보기' 버튼이 없어지거나 리뷰가 더 이상 늘어나지 않으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
            driver.execute_script("window.scrollBy(0, 500);")  # 스크롤을 조금씩 내려 모든 리뷰 로드

        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (202

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # 10개씩 리뷰를 수집하고 '더보기' 버튼 클릭
    while len(reviews) < total_reviews:
        # 리뷰 요소를 10개씩 재로드하여 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        
        for review_element in review_elements[len(reviews):len(reviews)+10]:  # 새로 10개씩 수집
            try:
                # 작성자 이름 (출력용)
                author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                # 리뷰 개수 및 사진 개수
                try:
                    review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                except NoSuchElementException:
                    review_count_text = "Unknown"

                # 리뷰 내용과 날짜
                try:
                    review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                    review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                    review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                except NoSuchElementException:
                    review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                review_key = (author_name, review_text, review_date, review_count_text)
                if review_key not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add(review_key)
                    print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                    break
            except NoSuchElementException:
                continue

        # '더보기' 버튼이 없어지거나 리뷰가 더 이상 늘어나지 않으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
            driver.execute_script("window.scrollBy(0, 500);")  # 스크롤을 조금씩 내려 모든 리뷰 로드

        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (202

# 중복방지 제대로 됨 위에꺼

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # 리뷰를 수집하고 '더보기' 버튼을 반복해서 누르기
    while len(reviews) < total_reviews:
        # 리뷰 요소들을 가져와 현재 리뷰 개수를 확인
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)

        # 현재 리뷰 개수와 수집된 리뷰 개수 비교
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        # 리뷰 개수가 일치하지 않으면 반복해서 리뷰를 수집
        if loaded_review_count > len(reviews):
            for review_element in review_elements[len(reviews):]:  # 새로운 리뷰만 수집
                try:
                    # 작성자 이름 (출력용)
                    author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                    # 리뷰 개수 및 사진 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                        review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                    # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                        break
                except NoSuchElementException:
                    continue

        # '더보기' 버튼이 없어지거나 리뷰가 더 이상 늘어나지 않으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기

        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # 리뷰를 수집하고 더보기 버튼이 사라질 때까지 반복
    while len(reviews) < total_reviews:
        # 리뷰 요소들을 가져와 현재 리뷰 개수를 확인
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)

        # 현재 리뷰 개수와 수집된 리뷰 개수 비교
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        # 새로운 리뷰가 로드될 때만 수집
        for review_element in review_elements[len(reviews):]:  # 새로운 리뷰만 수집
            try:
                # 작성자 이름 (출력용)
                author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                # 리뷰 개수 및 사진 개수
                try:
                    review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                except NoSuchElementException:
                    review_count_text = "Unknown"

                # 리뷰 내용과 날짜
                try:
                    review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                    review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                    review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                except NoSuchElementException:
                    review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                review_key = (author_name, review_text, review_date, review_count_text)
                if review_key not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add(review_key)
                    print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                    break
            except NoSuchElementException:
                continue

        # '더보기' 버튼이 없으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()
    last_loaded_count = 0

    # 리뷰를 수집하고 더보기 버튼이 사라질 때까지 반복
    while len(reviews) < total_reviews:
        # 리뷰 요소들을 가져와 현재 리뷰 개수를 확인
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)

        # 수집된 리뷰와 로드된 리뷰 개수를 비교
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        # 새로운 리뷰가 로드될 때만 수집
        if loaded_review_count > len(reviews):
            for review_element in review_elements[len(reviews):]:  # 새로운 리뷰만 수집
                try:
                    # 작성자 이름 (출력용)
                    author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                    # 리뷰 개수 및 사진 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                        review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                    # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                        break
                except NoSuchElementException:
                    continue

        # 스크롤을 하며 모든 리뷰가 로드되도록 유도
        if loaded_review_count == last_loaded_count:  # 새로 로드된 리뷰가 없으면 스크롤 조정
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정합니다.")
            driver.execute_script("window.scrollTo(0, 0);")  # 맨 위로 스크롤
            time.sleep(1)
            for scroll_pos in range(500, 3000, 500):  # 천천히 아래로 스크롤
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)
        
        last_loaded_count = loaded_review_count

        # '더보기' 버튼이 있으면 클릭하고, 없으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
        except (NoSuchElementException, TimeoutException):
            print("더보기 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로

In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()
    last_loaded_count = 0
    scroll_attempts = 0

    # 리뷰를 수집하고 더보기 버튼이 사라질 때까지 반복
    while len(reviews) < total_reviews:
        # 리뷰 요소들을 가져와 현재 리뷰 개수를 확인
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)

        # 수집된 리뷰와 로드된 리뷰 개수를 비교
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        # 새로운 리뷰가 로드될 때만 수집
        if loaded_review_count > len(reviews):
            for review_element in review_elements[len(reviews):]:  # 새로운 리뷰만 수집
                try:
                    # 작성자 이름 (출력용)
                    author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                    # 리뷰 개수 및 사진 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                        review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                    # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                        break
                except NoSuchElementException:
                    continue

        # 더 이상 새로운 리뷰가 로드되지 않으면 스크롤을 조정
        if loaded_review_count == last_loaded_count:
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정합니다.")
            scroll_attempts += 1
            driver.execute_script("window.scrollTo(0, 0);")  # 맨 위로 스크롤
            time.sleep(1)
            for scroll_pos in range(500, 3000, 500):  # 천천히 아래로 스크롤
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)
            if scroll_attempts > 5:  # 반복적인 시도 후에도 더 이상 로드가 안되면 중단
                print("더 이상 새로운 리뷰가 로드되지 않아 중단합니다.")
                break
        else:
            scroll_attempts = 0  # 새로운 리뷰가 로드되면 시도 횟수 초기화

        last_loaded_count = loaded_review_count

        # '더보기' 버튼이 있으면 클릭하고, 없으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
        except (NoSuchElementException, TimeoutException):
            print("더보기 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로

In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 활성화될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한
    print("총 리뷰 수:", total_reviews)

    reviews = []
    review_text_set = set()
    last_loaded_count = 0
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            review_text = "내용 없음"
                            review_date = "날짜 없음"

                    # 중복 방지
                    review_key = (author_name, review_text, review_date)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name}")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        last_loaded_count = loaded_review_count

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (2023년 9월 18일 월요일) - 작성자: lov****
리뷰 14: 몇일전 혼자먹

In [ ]:
//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[40]/div[6]/a

In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 활성화될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한
    print("총 리뷰 수:", total_reviews)

    reviews = []
    review_text_set = set()
    last_loaded_count = 0
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                # 새로운 예외 경로로 시도 (XPath 40번 위치)
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지
                    review_key = (author_name, review_text, review_date)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name}")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        last_loaded_count = loaded_review_count

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (2023년 9월 18일 월요일) - 작성자: lov****
리뷰 14: 몇일전 혼자먹

In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 활성화될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한
    print("총 리뷰 수:", total_reviews)

    reviews = []
    review_text_set = set()
    last_loaded_count = 0
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        last_loaded_count = loaded_review_count

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 51
로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일) - 작성자: lhj9382 (사진 11)
리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일) - 작성자: 솜사탕1387 (사진 2)
리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일) - 작성자: 꿀돼지새끼 (사진 380)
리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일) - 작성자: 무지개뜰땐무지개떡 (사진 1,815)
리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일) - 작성자: ㅋyle (사진 82)
리뷰 7: 굿 (2024년 3월 20일 수요일) - 작성자: 네이벙8 (사진 2)
리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일) - 작성자: 먹학박사 (사진 783)
리뷰 9: 양이 많아요 (2024년 2월 21일 수요일) - 작성자: 구미호75 (사진 26)
리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일) - 작성자: lhj9382 (사진 11)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일) - 작성자: Twinkle8681 (사진 512)
리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일) - 작성자: 초코초코초코초코초코초 (사진 94)
리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        total_reviews = min(total_reviews, 1000)  # 최대 1000개까지만 수집
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    reviews = []
    review_text_set = set()

    # 10개씩 리뷰를 수집하고 '더보기' 버튼 클릭
    while len(reviews) < total_reviews:
        # 리뷰 요소를 10개씩 재로드하여 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        
        for review_element in review_elements[len(reviews):len(reviews)+10]:  # 새로 10개씩 수집
            try:
                # 작성자 이름 (출력용)
                author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()

                # 리뷰 개수 및 사진 개수
                try:
                    review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                except NoSuchElementException:
                    review_count_text = "Unknown"

                # 리뷰 내용과 날짜
                try:
                    review_text_elements = review_element.find_elements(By.XPATH, './/div[5]/a[1]')
                    review_text = review_text_elements[0].text.strip() if review_text_elements else "내용 없음"
                    review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                except NoSuchElementException:
                    review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                    review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()

                # 중복 방지 및 저장 (없는 요소는 제외하여 키를 구성)
                review_key = (author_name, review_text, review_date, review_count_text)
                if review_key not in review_text_set:
                    reviews.append((review_date, review_text))
                    review_text_set.add(review_key)
                    print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                if len(reviews) >= total_reviews:  # 최대 수집 수 확인
                    break
            except NoSuchElementException:
                continue

        # '더보기' 버튼이 없어지거나 리뷰가 더 이상 늘어나지 않으면 종료
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)  # 페이지 로드 대기
            driver.execute_script("window.scrollBy(0, 500);")  # 스크롤을 조금씩 내려 모든 리뷰 로드

        except (NoSuchElementException, TimeoutException):
            print("하단 '더보기' 버튼이 없어졌습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 가져왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

# CSV 파일로 저장 (날짜와 리뷰 내용만 저장)
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


리뷰 탭 클릭 완료
총 리뷰 수: 223
리뷰 1: 음식은 매우 맛있었습니다
초리조 파니니!! 진짜 먹어본 파니니 중에 제일 바삭함 유지도 오래되면서 매콤하고. 소스를 묻혀먹으면 좀더 군침이 돌게 되어서 아주 맛있었고 토마토스프는 생각보단 묽었지만 그렇다고 맛이떨어지는 건 아니었습니다. ㅎㅎ
다만 이곳이 큐물러스가 맞는지 한참 고민하고 매장에 들어섰고(간판을 못찾겠음)
매장 한켠에 조그맣게 자리가 있는데
주문이 많이 많이 들어오더라구요. 그래서 많이 바쁘신데, 메뉴는 또 많이 바스락 거리며 떨어지는 재질이라 부스러기는 좀 관리가 어려워 보였습니다.
저는 제가 흘리거나 한 부스러기를 모두 치우고 일어서긴 했지만, 그렇지 못하는 분들도 있을테니 조금 관심가져 주시면 좋을 거 같습니다! (2024년 10월 26일 토요일) - 작성자: 하늘덕후 (사진 28)
리뷰 2: 오가다 저기가 뭐하는 곳인가,,,,,,,,,,,,,,,,,,
살펴보니 샌드위치 커피집^^

밥먹고 방문한 터라
커피만 시켰는데 커피 맛남.
건물 로비같은 곳에 무심히 꾸민 커피집 분위기
의외로 산만하지않고 대화하기 좋아씀^^
그리고 블루리본도 있고,,,,,

주변 프렌차이즈 커피집만 가보다
방문하니 신선쓰~~~
오가다 또 들릴 것 같음😁😁 (2024년 9월 10일 화요일) - 작성자: 외유내강369 (사진 1,983)
리뷰 3: 와 생각보다배불러요! 그리고 샐러드 파니니 파니니소스까지 다 맛있어요! 트러플 소스 완전 맛있게 먹었어요!! 추천합니다 (2024년 9월 10일 화요일) - 작성자: 블링블링66 (사진 191)
리뷰 4: 맛은 너무 좋았는데 좀 식어서 나와서 아쉬웠어요ㅜ 따뜻하게 먹고 싶었는데…! (2024년 9월 23일 월요일) - 작성자: 브루 (사진 69)
리뷰 5: 샌드위치가 맛있어요. (2024년 11월 10일 일요일) - 작성자: 샤벳37 (사진 62)
리뷰 6: 점심에 잠시 와서 커피랑 샐러드 먹었어용ㅎㅎㅎㅎㅎ 매장도 쾌적하구 좋아요 (2024년 8월 7일 수요일) - 작성자: 밍